In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Ruta del archivo Excel donde se guardarán los resultados
excel_path = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\PAMPA.xlsx"

# IP SOCKS5 específica
ip_socks5 = "socks5://199.188.93.214"  # Cambia esta IP a la que desees usar

# Función para realizar el scraping y extraer los nombres de los productos
def scrape_products():
    # Iniciar la lista de datos de productos
    product_data = []
    
    # Función para configurar el navegador con una IP SOCKS5 específica
    def get_driver_with_ip(ip):
        chrome_options = Options()
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument(f"--proxy-server={ip}")
        chrome_options.add_argument("--proxy-bypass-list=*")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        return driver

    # Página inicial
    page_number = 1
    while page_number <= 9:  # Cambiar la condición para detenerse en la página 9
        print(f"Procesando página {page_number}...")
        url = f'https://cl.puma.com/ni-os/categorias/zapatillas.html?p={page_number}'
        
        # Abrir la página de productos con una IP SOCKS5
        driver = get_driver_with_ip(ip_socks5)
        driver.get(url)
        driver.maximize_window()

        # Esperar a que los productos estén cargados
        WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-item")))

        # Encontrar los productos en la página
        product_items = driver.find_elements(By.CSS_SELECTOR, ".product-item")
        
        if not product_items:  # Si no hay productos, salimos del bucle
            print("No se encontraron productos en la página. Terminando el scraping.")
            break

        # Iterar sobre los elementos de los productos
        for index, product_item in enumerate(product_items):
            try:
                # Extraer la URL, nombre y URL de la imagen del producto
                product_url = product_item.find_element(By.CSS_SELECTOR, ".product-item__name").get_attribute('href')
                product_name = product_item.find_element(By.CSS_SELECTOR, ".product-item__name").text
                product_image_url = product_item.find_element(By.CSS_SELECTOR, ".product-item__img").get_attribute('src')

                # Extraer el precio original usando el atributo data-price-amount
                try:
                    original_price_element = product_item.find_element(By.CSS_SELECTOR, "span[data-price-amount]")
                    original_price = original_price_element.get_attribute("data-price-amount")
                except:
                    original_price = ""  # Dejar vacío si no se encuentra el precio original

                # Extraer el precio final (si está presente)
                try:
                    final_price_element = product_item.find_element(By.CSS_SELECTOR, ".product-item__price")
                    final_price = final_price_element.text.strip().replace("CLP", "").replace(".", "").strip()
                except:
                    final_price = ""  # Dejar vacío si no se encuentra el precio final

                # Imprimir los detalles del producto
                print(f"Producto {index + 1}: {product_name}")
                print(f"URL del Producto: {product_url}")
                print(f"URL de la Imagen: {product_image_url}")
                print(f"Precio Original: {original_price}")
                print(f"Precio Final: {final_price}")

                # Guardar la información del producto en la lista
                product_data.append({
                    "URL": product_url,
                    "Nombre del Producto": product_name,
                    "URL de la Imagen": product_image_url,
                    "Precio Original": original_price,
                    "Precio Final": final_price
                })

            except Exception as e:
                print(f"Error al procesar el producto {index + 1}: {e}")
        
        # Cerrar el navegador después de procesar la página actual
        driver.quit()

        # Incrementar el número de página para la siguiente iteración
        page_number += 1

    # Escribir los datos en el archivo Excel
    if product_data:
        try:
            # Verificar si el archivo Excel existe
            if os.path.exists(excel_path):
                try:
                    # Leer el archivo Excel existente
                    existing_df = pd.read_excel(excel_path, sheet_name="Hoja1")
                    # Concatenar los datos existentes con los nuevos datos
                    combined_df = pd.concat([existing_df, pd.DataFrame(product_data)], ignore_index=True)
                    # Escribir los datos combinados en el archivo Excel
                    combined_df.to_excel(excel_path, index=False, sheet_name="Hoja1")
                except Exception as e:
                    print(f"Error al leer o combinar datos con el archivo Excel: {e}")
            else:
                try:
                    # Si el archivo Excel no existe, crear el archivo
                    pd.DataFrame(product_data).to_excel(excel_path, index=False, sheet_name="Hoja1")
                except Exception as e:
                    print(f"Error al guardar los datos en el archivo Excel: {e}")
            
            print(f"Se han guardado {len(product_data)} productos en el archivo Excel.")
        except Exception as e:
            print(f"Error al guardar los datos en el archivo Excel: {e}")
    else:
        print("No se encontraron productos para guardar.")

# Ejecutar la función
scrape_products()


Procesando página 1...
Producto 1: Zapatillas de basketball MB.01 Thermal juveniles
URL del Producto: https://cl.puma.com/zapatillas-de-basketball-mb-01-thermal-juveniles-310509-01.html
URL de la Imagen: https://images.puma.com/image/upload/f_auto,q_auto,b_rgb:fafafa/global/310509/01/sv01/fnd/CHL/w/640/h/640/fmt/png
Precio Original: 119990
Precio Final: $119990
Producto 2: Zapatillas de basketball MB.03 Halloween juveniles
URL del Producto: https://cl.puma.com/zapatillas-de-basketball-mb-03-halloween-juveniles-311177-01.html
URL de la Imagen: https://images.puma.com/image/upload/f_auto,q_auto,b_rgb:fafafa/global/311177/01/sv01/fnd/CHL/w/640/h/640/fmt/png
Precio Original: 115990
Precio Final: $115990
Producto 3: Zapatillas de basketball MB.04 1Love juveniles
URL del Producto: https://cl.puma.com/zapatillas-de-basketball-mb-04-1love-juveniles-310533-01.html
URL de la Imagen: https://images.puma.com/image/upload/f_auto,q_auto,b_rgb:fafafa/global/310533/01/sv01/fnd/CHL/w/640/h/640/fmt/png
